### Progetto 8b
Luca Maria Lauricella 519997

Valerio Marini 512489

Repository del progetto:
https://github.com/lauriluca99/TGW-3D.jl

Documentazione del progetto:
https://lauriluca99.github.io/TGW-3D.jl

In [1]:
#Importing all the libraries used in the notebooks
using Profile
using ProfileView
using BenchmarkTools
using SparseArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
using NearestNeighbors
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

    function merge_vertices(
            V::Points, 
            EV::ChainOp, 
            FE::ChainOp, 
            [err=1e-4])
	
Rimuove i vertici congruenti ad un singolo rappresentatante, traduce i lati per tener 
conto della congruenza ed otteniene nuove facce congruenti.
#### Argomenti addizionali:
- `err`: Limite di errore massimo che si vuole utilizzare. Di Defaults a `1e-4`.

### Versione iniziale di merge_vertices

In [2]:
function merge_vertices(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)
    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)
    # remove vertices congruent to a single representative
    todelete = []
    i = 1
    for vi in 1:vertsnum
        if !(vi in todelete)
            nearvs = Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs = setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    for ei in 1:edgenum
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end
    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()
    for ei in 1:nedgenum
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    for e in 1:nedgenum
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
 
    visited = []
    function filter_fn(face)
 
        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)
 
        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end
 
    nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    for fi in 1:nfacenum
        for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Lar.Points(nV), nEV, nFE
 end

merge_vertices (generic function with 2 methods)

In [3]:
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
]

EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
])

FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

In [4]:
@btime merge_vertices(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)

  40.000 μs (920 allocations: 64.00 KiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 5, 1, 2, 6, 2, 3, 7, 3  …  12, 6, 9, 10, 7, 10, 11, 8, 11, 12], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  5, 6, 6, 6, 7, 7, 7, 8, 8, 8], Int8[-1, -1, -1, 1, -1, -1, 1, -1, -1, 1  …  -1, 1, 1, -1, 1, 1, -1, 1, 1, 1], 12, 8), sparse([1, 3, 1, 4, 1, 5, 1, 6, 3, 6  …  5, 6, 2, 3, 2, 4, 2, 5, 2, 6], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  8, 8, 9, 9, 10, 10, 11, 11, 12, 12], Int8[1, -1, 1, -1, 1, -1, -1, 1, 1, -1  …  -1, 1, -1, 1, -1, 1, -1, 1, 1, -1], 6, 12))

Si nota come i tipi delle variabili sono state già assegnati opportunamente in precedenza. Dunque non si ritiene necessario assegnarne di nuovi.

In [5]:
@code_warntype merge_vertices(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)

MethodInstance for merge_vertices(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Float64)
  from merge_vertices(V::Matrix, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, err) in Main at c:\Users\Valerio\Desktop\TGW-3D.jl\notebooks\merge_vertices.ipynb:1
Arguments
  #self#::Core.Const(merge_vertices)
  V@_2::Matrix{Float64}
  EV::SparseMatrixCSC{Int8, Int64}
  FE::SparseMatrixCSC{Int8, Int64}
  err::Float64
Locals
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  #10::var"#10#15"{SparseMatrixCSC{Int8, Int64}, Vector{Tuple{Int64, Int64}}, Vector{Int64}}
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  @_9::Union{Nothing, Tuple{Int64, Int64}}
  #9::var"#9#14"
  @_11::Union{Nothing, Tuple{Int64, Int64}}
  @_12::Union{Nothing, Tuple{Int64, Int64}}
  nFE::SparseMatrixCSC{Int8, Int64}
  nfacenum::Int64
  nfaces::Vector{Vector{Tuple{Int64, Int64}}}
  filter_fn::var"#filter_fn#18"{Vector{Any}}
  visited::Vector{Any}
  faces::Vector{Vector{Tuple{

)::Bool
│    %118 = Base.not_int(%117)::Bool
└───        goto #12 if not %118
10 ┄ %120 = @_9::Tuple{Int64, Int64}
│           (ei@_40 = Core.getfield(%120, 1))
│    %122 = Core.getfield(%120, 2)::Int64
│    %123 = nEV::SparseMatrixCSC{Int8, Int64}
│    %124 = ei@_40::Int64
│    %125 = Base.getindex(nedges, ei@_40)::Tuple{Int64, Int64}
│    %126 = Main.collect(%125)::Vector{Int64}
│    %127 = Base.dotview(%123, %124, %126)::Core.PartialStruct(SubArray{Int8, 1, SparseMatrixCSC{Int8, Int64}, Tuple{Int64, Vector{Int64}}, false}, Any[SparseMatrixCSC{Int8, Int64}, Tuple{Int64, Vector{Int64}}, Core.Const(0), Core.Const(0)])
│    %128 = Base.broadcasted(Base.identity, 1)::Core.Const(Base.Broadcast.Broadcasted(identity, (1,)))
│           Base.materialize!(%127, %128)
│           nEV
│    %131 = etuple2idx::Dict{Tuple{Int64, Int64}, Int64}
│    %132 = ei@_40::Int64
│    %133 = Base.getindex(nedges, ei@_40)::Tuple{Int64, Int64}
│           Base.setindex!(%131, %132, %133)
│           (@_9 = Bas

# Ottimizzazione

Inserendo la macro `@async` davanti agli opportuni cicli for, abbiamo ottenuto un codice che si comporta come una versione più veloce del codice precedente risparmiando circa un 30% del tempo di esecuzione.

In [6]:
function filter_fn(face)
    visited = []
    verts = []
    map(e->verts = union(verts, collect(e)), face)
    verts = Set(verts)
 
    if !(verts in visited)
        push!(visited, verts)
        return true
    end
    return false
end

filter_fn (generic function with 1 method)

In [7]:
function merge_vertices2(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)

    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)

    # remove vertices congruent to a single representative
    todelete = []
    i = 1

    for vi in 1:vertsnum #questo for non può essere parallelizzato
        if !(vi in todelete)
            nearvs = Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs = setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)

    for ei in 1:edgenum #questo for non può essere parallelizzato
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end

    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()

    for ei in 1:nedgenum #questo ciclo non può essere parallelizzato
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    
    #questo ciclo può essere parallelizzato ma non cambia il tempo per un numero di edge piccolo
    @async for e in 1:nedgenum 
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
    @inbounds nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    @async for fi in 1:nfacenum
        @async for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Lar.Points(nV), nEV, nFE
 end

merge_vertices2 (generic function with 2 methods)

In [8]:
@btime merge_vertices2(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)

  35.300 μs (779 allocations: 56.56 KiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 5, 1, 2, 6, 2, 3, 7, 3  …  12, 6, 9, 10, 7, 10, 11, 8, 11, 12], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  5, 6, 6, 6, 7, 7, 7, 8, 8, 8], Int8[-1, -1, -1, 1, -1, -1, 1, -1, -1, 1  …  -1, 1, 1, -1, 1, 1, -1, 1, 1, 1], 12, 8), sparse([1, 3, 1, 4, 1, 5, 1, 6, 3, 6  …  5, 6, 2, 3, 2, 4, 2, 5, 2, 6], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  8, 8, 9, 9, 10, 10, 11, 11, 12, 12], Int8[1, -1, 1, -1, 1, -1, -1, 1, 1, -1  …  -1, 1, -1, 1, -1, 1, -1, 1, 1, -1], 6, 12))

In [9]:
@benchmark merge_vertices(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-4)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  39.200 μs …  13.339 ms  ┊ GC (min … max):  0.00% … 99.16%
 Time  (median):     49.400 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   74.714 μs ± 368.686 μs  ┊ GC (mean ± σ):  18.01% ±  3.68%

  ▄█▇▆▅▅▄▄▄▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁ ▁  ▁   ▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁ ▁         ▂
  ████████████████████████████████████████████████████████▆█▆▇ █
  39.2 μs       Histogram: log(frequency) by time       141 μs <

 Memory estimate: 64.00 KiB, allocs estimate: 920.

In [10]:
@benchmark merge_vertices2(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-4)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  34.700 μs … 94.229 ms  ┊ GC (min … max):  0.00% … 99.75%
 Time  (median):     54.600 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   99.073 μs ±  1.576 ms  ┊ GC (mean ± σ):  27.47% ±  1.73%

   █▅▁                                                         
  ▅████▇▆▆▅▄▄▃▃▂▂▂▂▂▂▂▂▃▂▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  34.7 μs         Histogram: frequency by time         218 μs <

 Memory estimate: 56.56 KiB, allocs estimate: 779.